<a href="https://colab.research.google.com/github/tproffen/ORCSGirlsPython/blob/master/DoodleMining/QuickDrawDemos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/tproffen/ORCSGirlsPython/blob/master/Images/Logo.png?raw=1" width="10%" align="right" hpsace="50">

# Doodle Data Mining

## Demonstration of using the Quickdraw dataset

* Quickdraw: https://quickdraw.withgoogle.com/ 
* Quickdraw dataset: https://quickdraw.withgoogle.com/data
* Quickdraw API: https://quickdraw.readthedocs.io/en/latest/index.html

* All valid categories are here : https://github.com/googlecreativelab/quickdraw-dataset/blob/master/categories.txt

**More reading**
* https://opensource.googleblog.com/2018/11/introducing-web-component-and-data-api-for-quick-draw.html

Here are the completed examples from the activity notebooks. For details refer to the activity notebooks.

## Run these calls first

The two cells below need to be executed first. They install the Quickdraw API and load the needed Python modules.

In [ ]:
!curl -s -o setup.sh https://raw.githubusercontent.com/tproffen/ORCSGirlsPython/master/DoodleMining/Helpers/setup_activity1.sh
!bash setup.sh

from Helpers.helpers import *

# Activity 1
## Retrieving drawings
### Information about a single doodle

In [ ]:
what="spider"
doodles = QuickDrawDataGroup(what)

In [ ]:
doodle = doodles.get_drawing()

In [ ]:
dir(doodle)

In [ ]:
print (doodle.countrycode)
print (datetime.fromtimestamp(doodle.timestamp))
print (len(doodle.strokes))

### Drawing a single doodle

In [ ]:
what="firetruck"
doodles = QuickDrawDataGroup(what)

In [ ]:
doodle = doodles.get_drawing()
image = doodle.get_image(stroke_color=(255, 0, 0), stroke_width=3, bg_color=(255, 255, 0))
plot_image(image)

print ("Country: ",doodle.countrycode)
print ("Date",datetime.fromtimestamp(doodle.timestamp))
print ("Number of strokes: ",len(doodle.strokes))

## Retrieving doodles by country

In [ ]:
what = "bread"
doodles = QuickDrawDataGroup(what, max_drawings=5000, recognized=True)

In [ ]:
# List of countries
countries = ["US", "FR"]
# How many columns (or doodles for each country)
cols = 5  

plt.figure(figsize=(20, 15)) # Makes the plot bigger

index=1
for country in countries:
  # This gets drawings for the given country code
  doodles_country=doodles.search_drawings(countrycode=country)
  # We shuffle the returned list, so we get different drawings when re-running
  random.shuffle(doodles_country)

  for col in range(cols):
    img = doodles_country[col].get_image()
    plt.subplot(len(countries), cols, index)
    plt.imshow(img)
    plt.title(country)
    plt.axis('off')
    index+=1;

plt.show()

# Activity 2
## Let's see how a doodle was drawn


In [ ]:
what="sun"
doodles = QuickDrawDataGroup(what)

In [ ]:
doodle = doodles.get_drawing()

# print doodle
print (doodle)
# print the number of lines
print (len(doodle.strokes))

# Get the points for the first line line
line=doodle.strokes[0]
print (line)
# now get the x and y of the first point (there are a tuple, so you can do (x,y) = ..)
(x,y)=line[0]
print(x,y)

In [ ]:
initializeTurtle()
showturtle()

for stroke in doodle.strokes:
  penup()
  for x,y in stroke:
    goto(300+x,200+y)
    pendown()

show()

## Project: How do people draw circles

In [ ]:
def winding(stroke):
  if (len(stroke) < 3):
    return 0
  else:
    (xa,ya) = stroke[0]    
    (xb,yb) = stroke[1]    
    (xc,yc) = stroke[2]
    return (xb*yc+xa*yb+ya*xc) - (ya*xb+yb*xc+xa*yc)

In [ ]:
what = "circle"
all = QuickDrawDataGroup(what, max_drawings=50000, recognized=True)

In [ ]:
countries = ["US", "JP", "DE", "TW"]
countries_cw = []

for country in countries:
  doodles=all.search_drawings(countrycode=country)
  count=0
  for doodle in doodles:
    w=winding(doodle.strokes[-1])
    if (w>0):
      count +=1
  percent_cw=100*count/len(doodles)
  countries_cw.append(percent_cw)

print(countries, countries_cw)

In [ ]:
plot(countries_cw, countries)

In [ ]:
# Let create two functions, so we can reuse them as needed.

# Function sort_doodles 
# Sort the doodles into cw and ccw

def sort_doodles(doodles):
  cw = []
  ccw = []

  for doodle in doodles[:30]:
    w=winding(doodle.strokes[-1])
    if (w>0):
      cw.append(doodle.strokes[-1])
    else:
      ccw.append(doodle.strokes[-1])
  return(cw, ccw)

# Function draw_doodles
# Draw doodles using the turtle. Needs to be called
# after initializeTurtle.

def draw_doodles(x_start,y_start,col,strokes):
  for stroke in strokes:
    color(col)
    penup()
    for x,y in stroke:
      goto(x_start+x,y_start+y)
      pendown()


# Making the master plot using our new functions

initializeTurtle()

doodles=all.search_drawings(countrycode="US")
(us_cw, us_ccw) = sort_doodles(doodles)
draw_doodles( 10, 10,'red',us_cw)
draw_doodles(400, 10,'blue',us_ccw)
print ("US: cw:",len(us_cw)," ccw:",len(us_ccw))

doodles=all.search_drawings(countrycode="JP")
(jp_cw, jp_ccw) = sort_doodles(doodles)
draw_doodles( 10,300,'red',jp_cw)
draw_doodles(400,300,'blue',jp_ccw)
print ("JP: cw:",len(jp_cw)," ccw:",len(jp_ccw))

show()